# Association of chrM2158T>C variant with PD risk in AllOfUs
* **Project:** Mitochondrial 2158T>C variant in PD
* **Version:** Python/3.9
* **Status:** COMPLETE
* **Last Updated:** 07-APRIL-2024

### Notebook Overview
Querying AllOfUs to extract neurologically healthy controls > 70 years CRAM files, merge them, use Mutect2 and PLINK2 for analysis

In [3]:
import pandas
import os

# This query represents dataset "pd_control70" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_18972590_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 
                ) 
                AND cb_search_person.person_id IN (
                    SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE,dob, YEAR) - IF(EXTRACT(MONTH 
                    FROM
                        dob)*100 + EXTRACT(DAY 
                    FROM
                        dob) > EXTRACT(MONTH 
                    FROM
                        CURRENT_DATE)*100 + EXTRACT(DAY 
                    FROM
                        CURRENT_DATE),
                        1,
                        0) BETWEEN 70 AND 120 
                        AND NOT EXISTS ( SELECT
                            'x' 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d 
                        WHERE
                            d.person_id = p.person_id) ) 
                        AND cb_search_person.person_id NOT IN (SELECT
                            criteria.person_id 
                        FROM
                            (SELECT
                                DISTINCT person_id,
                                entry_date,
                                concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                            WHERE
                                (
                                    concept_id IN (
                                        SELECT
                                            DISTINCT c.concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                        JOIN
                                            (
                                                select
                                                    cast(cr.id as string) as id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                WHERE
                                                    concept_id IN (4086181, 376337) 
                                                    AND full_text LIKE '%_rank1]%'
                                            ) a 
                                                ON (
                                                    c.path LIKE CONCAT('%.',
                                                a.id,
                                                '.%') 
                                                OR c.path LIKE CONCAT('%.',
                                                a.id) 
                                                OR c.path LIKE CONCAT(a.id,
                                                '.%') 
                                                OR c.path = a.id) 
                                            WHERE
                                                is_standard = 1 
                                                AND is_selectable = 1
                                            ) 
                                            AND is_standard = 1 
                                    )
                                ) criteria 
                            ) 
                            AND cb_search_person.person_id NOT IN (
                                SELECT
                                    criteria.person_id 
                                FROM
                                    (SELECT
                                        DISTINCT person_id,
                                        entry_date,
                                        concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                    WHERE
                                        (
                                            concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (1740714) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (1740915) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (1740796) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (43528614) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (43528690) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (1740817) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (43528397) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (1740757) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (43529225) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (43529248) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (43529256) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (596893) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (836761) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (43529691) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (836762) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (1740697) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (1740845) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (1740764) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (903079) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (903096) 
                                            OR  concept_id IN (43529272) 
                                            AND is_standard = 0  
                                            AND  value_source_concept_id IN (903087)
                                        )) criteria ) 
                            )"""

dataset_18972590_person_df = pandas.read_gbq(
    dataset_18972590_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_18972590_person_df.head(5)
# To write the DataFrame to a CSV file:
dataset_18972590_person_df.to_csv("mar6_controls70_person_df.csv", index=False)

Downloading:   0%|          | 0/24851 [00:00<?, ?rows/s]

In [4]:
import pandas
import os

# This query represents dataset "pd_control70" for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_18972590_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (
                43529272, 836812
            )
        )  
        AND (
            answer.PERSON_ID IN (
                SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (
                        SELECT
                            person_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                        WHERE
                            has_whole_genome_variant = 1 
                    ) 
                    AND cb_search_person.person_id IN (
                        SELECT
                            person_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                        WHERE
                            DATE_DIFF(CURRENT_DATE,dob, YEAR) - IF(EXTRACT(MONTH 
                        FROM
                            dob)*100 + EXTRACT(DAY 
                        FROM
                            dob) > EXTRACT(MONTH 
                        FROM
                            CURRENT_DATE)*100 + EXTRACT(DAY 
                        FROM
                            CURRENT_DATE),
                            1,
                            0) BETWEEN 70 AND 120 
                            AND NOT EXISTS ( SELECT
                                'x' 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.death` d 
                            WHERE
                                d.person_id = p.person_id) ) 
                            AND cb_search_person.person_id NOT IN (SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        concept_id IN (
                                            SELECT
                                                DISTINCT c.concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                            JOIN
                                                (
                                                    select
                                                        cast(cr.id as string) as id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                    WHERE
                                                        concept_id IN (4086181, 376337) 
                                                        AND full_text LIKE '%_rank1]%'
                                                ) a 
                                                    ON (
                                                        c.path LIKE CONCAT('%.',
                                                    a.id,
                                                    '.%') 
                                                    OR c.path LIKE CONCAT('%.',
                                                    a.id) 
                                                    OR c.path LIKE CONCAT(a.id,
                                                    '.%') 
                                                    OR c.path = a.id) 
                                                WHERE
                                                    is_standard = 1 
                                                    AND is_selectable = 1
                                                ) 
                                                AND is_standard = 1 
                                        )
                                    ) criteria 
                                ) 
                                AND cb_search_person.person_id NOT IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (1740714) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (1740915) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (1740796) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (43528614) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (43528690) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (1740817) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (43528397) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (1740757) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (43529225) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (43529248) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (43529256) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (596893) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (836761) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (43529691) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (836762) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (1740697) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (1740845) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (1740764) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (903079) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (903096) 
                                                OR  concept_id IN (43529272) 
                                                AND is_standard = 0  
                                                AND  value_source_concept_id IN (903087)
                                            )) criteria ) 
                                )
                            )"""

dataset_18972590_survey_df = pandas.read_gbq(
    dataset_18972590_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_18972590_survey_df.head(5)
# To write the DataFrame to a CSV file:
dataset_18972590_survey_df.to_csv("mar6_controls70_survey_df.csv", index=False)

Downloading:   0%|          | 0/12488 [00:00<?, ?rows/s]

In [5]:
import pandas
import os

# This query represents dataset "pd_control70" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
dataset_18972590_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN  (
                    SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (
                            select
                                cast(cr.id as string) as id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                            WHERE
                                concept_id IN (
                                    376337, 381270
                                ) 
                                AND full_text LIKE '%_rank1]%'
                        ) a 
                            ON (
                                c.path LIKE CONCAT('%.',
                            a.id,
                            '.%') 
                            OR c.path LIKE CONCAT('%.',
                            a.id) 
                            OR c.path LIKE CONCAT(a.id,
                            '.%') 
                            OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1
                        )
                )  
                AND (
                    c_occurrence.PERSON_ID IN (
                        SELECT
                            distinct person_id  
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                        WHERE
                            cb_search_person.person_id IN (
                                SELECT
                                    person_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                WHERE
                                    has_whole_genome_variant = 1 
                            ) 
                            AND cb_search_person.person_id IN (
                                SELECT
                                    person_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                WHERE
                                    DATE_DIFF(CURRENT_DATE,dob, YEAR) - IF(EXTRACT(MONTH 
                                FROM
                                    dob)*100 + EXTRACT(DAY 
                                FROM
                                    dob) > EXTRACT(MONTH 
                                FROM
                                    CURRENT_DATE)*100 + EXTRACT(DAY 
                                FROM
                                    CURRENT_DATE),
                                    1,
                                    0) BETWEEN 70 AND 120 
                                    AND NOT EXISTS ( SELECT
                                        'x' 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.death` d 
                                    WHERE
                                        d.person_id = p.person_id) ) 
                                    AND cb_search_person.person_id NOT IN (SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                concept_id IN (4086181, 376337) 
                                                                AND full_text LIKE '%_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            is_standard = 1 
                                                            AND is_selectable = 1
                                                        ) 
                                                        AND is_standard = 1 
                                                )
                                            ) criteria 
                                        ) 
                                        AND cb_search_person.person_id NOT IN (
                                            SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (1740714) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (1740915) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (1740796) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (43528614) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (43528690) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (1740817) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (43528397) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (1740757) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (43529225) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (43529248) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (43529256) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (596893) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (836761) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (43529691) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (836762) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (1740697) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (1740845) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (1740764) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (903079) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (903096) 
                                                        OR  concept_id IN (43529272) 
                                                        AND is_standard = 0  
                                                        AND  value_source_concept_id IN (903087)
                                                    )) criteria ) 
                                        )
                                    )) c_occurrence 
                            LEFT JOIN
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                                    ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
                            LEFT JOIN
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                                    ON c_occurrence.condition_type_concept_id = c_type.concept_id 
                            LEFT JOIN
                                `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                                    ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
                            LEFT JOIN
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                                    ON v.visit_concept_id = visit.concept_id 
                            LEFT JOIN
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                                    ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
                            LEFT JOIN
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                                    ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

dataset_18972590_condition_df = pandas.read_gbq(
    dataset_18972590_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_18972590_condition_df.head(5)
# To write the DataFrame to a CSV file:
dataset_18972590_condition_df.to_csv("mar6_controls70_condition_df.csv", index=False)

Downloading: 0rows [00:00, ?rows/s]

In [2]:
#Include only European samples
ancestry_pred_path = "gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv"
os.environ['ancestry_pred_path'] = ancestry_pred_path
!gsutil -u $GOOGLE_PROJECT cat $ancestry_pred_path | cut -f1,2 | grep eur > eur_ancestry
!grep -v person mar6_controls70_person_df.csv | cut -f1 -d "," > list_of_all_controlsover70

In [ ]:
#Control CRAM streaming
!gsutil -u $GOOGLE_PROJECT cp gs://${ALLOFUS}/v7/wgs/cram/manifest.csv .

In [ ]:
## Package Import
import sys
import os 
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
##Ensuring dsub is up to date
!pip3 install --upgrade dsub

In [ ]:
# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env JOB_ID={LINE_COUNT_JOB_ID}
## Defining necessary pathways
my_bucket = os.environ['WORKSPACE_BUCKET']

In [ ]:
mkdir CRAM_Streaming_control
!cp manifest_control.csv CRAM_Streaming_control/.
cram_manifest = pd.read_csv('CRAM_Streaming_control/manifest_control.csv')
cram_manifest['cram_uri'].iloc[0]

In [ ]:
## Setting for running dsub jobs
pd.set_option('display.max_colwidth', 0)

In [ ]:
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')

# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env USER_NAME={USER_NAME}
## MODIFY FOR FULL DATA RUN
# Use hyphens, not whitespace since it will become part of the bucket path.
JOB_NAME='CRAM_Stream_Test_ctrl'

# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env JOB_NAME={JOB_NAME}

In [ ]:
## Analysis Results Folder
line_count_results_folder = os.path.join(
    os.getenv('WORKSPACE_BUCKET'),
    'dsub',
    'results',
    JOB_NAME,
    USER_NAME,
    datetime.now().strftime('%Y%m%d'))

line_count_results_folder

In [ ]:
## Where the output files will go
output_files = os.path.join(line_count_results_folder, "results")
print(output_files)
OUTPUT_FILES = output_files


# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env OUTPUT_FILES={OUTPUT_FILES}

In [ ]:
cram_count = len(cram_manifest)
cram_count
jobs = cram_count/50
jobs

In [ ]:
##Creating batches of 10 will be larger for samples
##Crams 2-101
##For full just making a df with all crams
test_crams = cram_manifest['cram_uri'].iloc[0:]
test_crams

In [ ]:
## Made a batch folder
!mkdir CRAM_Streaming_control/batches
!realpath CRAM_Streaming_control/

In [ ]:
## Splitting into ten files with ten cram pathways per 
## Copy batch realpath output above into '' in the to_csv command being sure to keep the single quotes
## At the end of the path, be sure to add /cram_batch_{id}.txt 
for id, test_crams_i in  enumerate(np.array_split(test_crams, 30)):
    test_crams_i.to_csv('/home/jupyter/workspaces/neurologicalgenescreening/CRAM_Streaming_control/batches/cram_batch_{id}.txt'.format(id=id), index=False, header=None)

In [ ]:
## Coping batches to user directory
!gsutil -m cp CRAM_Streaming_control/batches/* {my_bucket}/dsub/final_batches/

In [ ]:
!gsutil ls {my_bucket}/dsub/final_batches/*.txt > AoU_batches_pdcontrol.txt

In [ ]:
## Move batch file
## Use realpath output above again in this command:
!mv AoU_batches_pdcontrol.txt /home/jupyter/workspaces/neurologicalgenescreening/CRAM_Streaming_control/

In [ ]:
!realpath CRAM_Streaming_control/AoU_batches_pdcontrol.txt

In [ ]:
%%writefile ~/printreads_depth_CRAM_Stream.sh

set -o pipefail
set -o errexit

# ---------Required Inputs---------
# aou_crams - A .txt file containing gs:// paths to cram samples.

# Given a .txt file - get X samples.
# For parallel submissions:
# - Use a different .txt file per submission.
# - Each .txt file can contain a different number of lines
aou_crams_len=$(wc -l < ${aou_crams})
echo "Samples in cramlist: ${aou_crams_len}"

# ---------Required Output---------
#filtered_cram_output

echo "GOOGLE_PROJECT: ${GOOGLE_PROJECT}"
echo "OUTPUT_PATH: ${OUTPUT_PATH}"
echo "ref_dict: ${ref_dict}"
echo "ref_fai: ${ref_fai}"
echo "ref_fasta: ${ref_fasta}"

# Perform runs for x samples.
for (( i=1; i<$aou_crams_len+1; i++ ));
do

    # These change per iteration
    export aou_cram_reads=$(sed "${i}!d;q" "${aou_crams}")   # gs:// path to a cram sample
    export aou_cram_reads_name=`basename ${aou_cram_reads}`  # file_name.cram
    export aou_cram_reads_prefix="${aou_cram_reads_name%.*}" # file_name
    echo "aou_cram_reads: ${aou_cram_reads}"
    echo "aou_cram_reads_name: ${aou_cram_reads_name}"
    echo "aou_cram_reads_prefix: ${aou_cram_reads_prefix}"

    # ----------------------------------WORKFLOW----------------------------------

    # Stream CRAM found at gs:// path into a new, smaller CRAM based on genomic intervals given with -L.
    /gatk/gatk PrintReads \
        -I ${aou_cram_reads} \
        -L "chrM" \
        -R "${ref_fasta}" \
        -O "${aou_cram_reads_prefix}_mt_control.cram" \
        --gcs-project-for-requester-pays ${GOOGLE_PROJECT} \
        --cloud-prefetch-buffer 0 --cloud-index-prefetch-buffer 0

    # Outputs
    export igk_depth_cram="${aou_cram_reads_prefix}_mt_control.cram"
    echo "igk_depth_cram: ${igk_depth_cram}"

    # Disk space
    echo "Disk space taken up so far:"
    du -d 1 -h
    echo "${i} run(s) finished."

    # Move results to output directory
    mv ${igk_depth_cram} ${OUTPUT_PATH}
done

In [ ]:
!gsutil cp /home/jupyter/printreads_depth_CRAM_Stream.sh {my_bucket}/dsub/scripts/

In [ ]:
# Copy this cell's output to the BASH_SCRIPT variable in cell 33, the dsub command below
!gsutil ls {my_bucket}/dsub/scripts/

In [ ]:
!cp CRAM_Streaming_control/AoU_batches_pdcontrol.txt .

In [ ]:
%%bash --out LINE_COUNT_JOB_ID

# Get a shorter username to leave more characters for the job name.
DSUB_USER_NAME="$(echo "${OWNER_EMAIL}" | cut -d@ -f1)"

# For AoU RWB projects network name is "network".
AOU_NETWORK=network
AOU_SUBNETWORK=subnetwork

# Get all cramlists
bashArray=()

## ------------------------------------------------ MAKE CHANGES HERE ------------------------------------------
#Change the 'done < test_cram_batch.txt' to 'done < AoU_v7_batches.txt' if you want to run across all batches
while read line; do
  bashArray+=($line)
done < AoU_batches_pdcontrol.txt
## -------------------------------------------------------------------------------------------------------------

# Length of entire array
len_bashArray=${#bashArray[@]}

LOWER=0
UPPER=$len_bashArray
MACHINE_TYPE="n2-standard-4"
## ------------------------------------------------ MAKE CHANGES HERE ------------------------------------------
DATE=20230301
BASH_SCRIPT="${WORKSPACE_BUCKET}/dsub/scripts/printreads_depth_CRAM_Stream.sh"
## -------------------------------------------------------------------------------------------------------------

for ((batch=$LOWER;batch<$UPPER;batch+=1))
do
    dsub \
        --provider google-cls-v2 \
        --user-project "${GOOGLE_PROJECT}"\
        --project "${GOOGLE_PROJECT}"\
        --network "${AOU_NETWORK}" \
        --subnetwork "${AOU_SUBNETWORK}" \
        --service-account "$(gcloud config get-value account)" \
        --user "${DSUB_USER_NAME}" \
        --regions us-central1 \
        --logging "${WORKSPACE_BUCKET}/dsub/logs/{job-name}/{user-id}/${DATE}/${bashArray[batch]}.log" \
        "$@" \
        --preemptible \
        --boot-disk-size 100 \
        --machine-type ${MACHINE_TYPE} \
        --disk-size 100 \
        --name "${JOB_NAME}" \
        --script "${BASH_SCRIPT}" \
        --image 'gcr.io/bick-aps2/briansha/pileup_preprocessing:latest' \
        --env GOOGLE_PROJECT=${GOOGLE_PROJECT} \
        --input ref_dict="${WORKSPACE_BUCKET}/data/Homo_sapiens_assembly38.dict" \
        --input ref_fai="${WORKSPACE_BUCKET}/data/Homo_sapiens_assembly38.fasta.fai" \
        --input ref_fasta="${WORKSPACE_BUCKET}/data/Homo_sapiens_assembly38.fasta" \
        --input aou_crams="${bashArray[batch]}" \
        --output-recursive OUTPUT_PATH="${OUTPUT_FILES}/${batch}"
done

In [ ]:
!for i in {1..29} ; do  gsutil -o 'GSUtil:parallel_thread_count=20' -m cp -r ${WORKSPACE_BUCKET}/dsub/results/CRAM_Stream_Test_ctrl/fulyaakcimen/20240303/results/$i/wgs_*_mt_control.cram . ; done
!for i in 0 ; do  gsutil -o 'GSUtil:parallel_thread_count=20' -m cp -r ${WORKSPACE_BUCKET}/dsub/results/CRAM_Stream_Test_ctrl/fulyaakcimen/20240303/results/$i/wgs_*_mt_control.cram . ; done

In [ ]:
!for i in `cat control_crams` ; do samtools index $i ; done

In [ ]:
!mkdir controls

In [ ]:
!for i in `cat control_crams` ; do gatk --java-options "-Xmx10G" Mutect2 -R Homo_sapiens_assembly38.fasta -L chrM --mitochondria-mode -I $i -O controls/"$i".vcf.gz ; done

In [ ]:
!bcftools merge controls/*gz -Oz -o controls_chr2158.vcf.gz --force-samples -r chrM:2157-2159

In [ ]:
!plink2 --vcf controls_chr2158.vcf.gz --make-bed --double-id --out controls_chr2158 \
--chr MT --from-bp 2158 --to-bp 2158

In [ ]:
!plink --bfile controls_chr2158 --fill-missing-a2 --make-bed --out PDfiltecontrols